## **Cryptocurrency Market Direction Prediction**

This notebook provides a method for **predicting the market direction of cryptocurrencies** using a pre-trained machine learning model. It loads a classification model stored as a Pickle file and financial data from a CSV file. For each requested cryptocurrency, it performs a prediction based on recent market characteristics and determines whether the consolidated trend over a given period is bullish or bearish.

## Importing packages

In [ ]:
!pip install fastapi
import pandas as pd
import pickle
from fastapi import HTTPException

## Direction prediction

In [ ]:
class DirectionRequest:
    def __init__(self, cryptos, period):
        self.cryptos = cryptos
        self.period = period

def predict_direction(request: DirectionRequest):
    """
    Prédit la direction consolidée du marché pour plusieurs cryptos sur une période spécifiée.
    """
    try:
        # Charger le modèle et les données
        with open("/content/direction_model.pkl", "rb") as model_file:
            model = pickle.load(model_file)
        df = pd.read_csv("/content/feature_data.csv")

        # Vérification des colonnes nécessaires
        required_columns = {'Crypto', 'Close', 'Open', 'High', 'Low', 'Volume'}
        if not required_columns.issubset(df.columns):
            raise HTTPException(status_code=400, detail="Les données ne contiennent pas les colonnes nécessaires.")

        predictions = {}

        for crypto in request.cryptos:
            # Filtrer pour la crypto spécifiée
            crypto_df = df[df['Crypto'] == crypto]
            if crypto_df.empty:
                predictions[crypto] = "No Data"
                continue

            # Utiliser les dernières données correspondant à la période spécifiée
            crypto_df = crypto_df.tail(request.period)
            if crypto_df.empty:
                predictions[crypto] = "Not Enough Data"
                continue

            # Préparer les données pour la prédiction
            features = ['Open', 'High', 'Low', 'Close', 'Volume']
            X = crypto_df[features].values

            # Faire des prédictions
            predicted_directions = model.predict(X)
            directions = ["Up" if direction == 1 else "Down" for direction in predicted_directions]

            # Consolider la direction pour la période
            consolidated_direction = "Up" if directions.count("Up") > directions.count("Down") else "Down"
            predictions[crypto] = consolidated_direction

        return {"predictions": predictions}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


HTTPException: 500: invalid load key, '\x01'.

Test with all cryptocurrencies

In [ ]:
request_example = DirectionRequest(cryptos=["BTC", "ETH"], period=7)
predict_direction(request_example)